In [1]:
from matplotlib import pyplot as plt
import h5py
import numpy as np
import matplotlib.gridspec as gridspec
from scipy import spatial
from tqdm import *
import random

cdm = h5py.File('Data/COLOR_CDM_DM_subHaloes_z0.00_v2.hdf5', 'r')
wdm = h5py.File('Data/COLOR_WDM_DM_subHaloes_z0.00_v2.hdf5', 'r')

cdm_galaxy = h5py.File('Data\COLOR_CDM_galaxies.hdf5', 'r')
wdm_galaxy = h5py.File('Data\COLOR_WDM_galaxies.hdf5', 'r')

print(cdm,wdm)

<HDF5 file "COLOR_CDM_DM_subHaloes_z0.00_v2.hdf5" (mode r)> <HDF5 file "COLOR_WDM_DM_subHaloes_z0.00_v2.hdf5" (mode r)>


# Find all galaxies that are as heavy as the Milky Way
### Access the masses and positions of CDM and WDM galaxies

In [2]:
# Access mass/position for wdm/cdm
print(list(cdm_galaxy.keys()),list(wdm_galaxy.keys()))

cdm_galaxy_color = cdm_galaxy[list(cdm_galaxy.keys())[0]][:]
cdm_galaxy_position = cdm_galaxy[list(cdm_galaxy.keys())[1]][:]
cdm_galaxy_mass = cdm_galaxy[list(cdm_galaxy.keys())[3]][:]

wdm_galaxy_color = wdm_galaxy[list(wdm_galaxy.keys())[0]][:]
wdm_galaxy_position = wdm_galaxy[list(cdm_galaxy.keys())[1]][:]
wdm_galaxy_mass = wdm_galaxy[list(wdm_galaxy.keys())[3]][:]

cdm_galaxy_color, cdm_galaxy_position, cdm_galaxy_mass, wdm_galaxy_color, wdm_galaxy_position, wdm_galaxy_mass

['Colour', 'GalaxyPos', 'IsCentral', 'StellarMass'] ['Colour', 'GalaxyPos', 'IsCentral', 'StellarMass']


(array([0.32435036, 0.58950651, 0.32047844, ..., 0.58985472, 0.3150177 ,
        0.60267353]),
 array([[17.43208122, 80.25539398, 54.00876999],
        [17.43210793, 80.22522736, 53.97425079],
        [19.06316185, 80.02703857, 54.18655014],
        ...,
        [67.22305298, 64.69418335, 36.19010544],
        [ 7.28926134, 78.05925751, 49.41135025],
        [ 6.75961494, 77.04682159, 48.55372238]]),
 array([2.37800096e+08, 1.14038586e+03, 4.39743280e+07, ...,
        6.31516409e+00, 3.49365275e+06, 2.19290527e+04]),
 array([0.29584217, 0.29859257, 0.59355354, ..., 0.32156372, 0.59656096,
        0.32803154]),
 array([[17.61932182, 79.06235504, 52.68593979],
        [18.65647697, 80.58162689, 52.30108261],
        [18.58250046, 80.91363525, 52.45199203],
        ...,
        [76.68208313, 28.50509262, 59.96631622],
        [33.62237167,  9.74790382,  4.82571173],
        [52.58901978, 44.79496765, 78.47211456]]),
 array([3873670.  , 5328977.  , 1002360.25, ..., 2697151.5 ,  542400.5 ,


### Find all central galaxies within the mass range $2-6*10^{10}$ M$_{\odot}$

In [3]:
# get the indices
cdm_MW_galaxy_ind = np.where((cdm_galaxy_mass>2*10**10) & (cdm_galaxy_mass<6*10**10) & (cdm_galaxy['IsCentral'][:]==1))[0]
wdm_MW_galaxy_ind = np.where((wdm_galaxy_mass>2*10**10) & (wdm_galaxy_mass<6*10**10) & (wdm_galaxy['IsCentral'][:]==1))[0]

In [4]:
# get the positions
cdm_MW_galaxy_position = cdm_galaxy_position[cdm_MW_galaxy_ind]
wdm_MW_galaxy_position = wdm_galaxy_position[wdm_MW_galaxy_ind]

### Set a minimum mass of galaxies to search through

In [5]:
# get the indices
m_min=0
cdm_galaxy_filtered_ind = np.where(cdm_galaxy_mass > m_min)[0]
wdm_galaxy_filtered_ind = np.where(wdm_galaxy_mass > m_min)[0]

In [6]:
# get the positions
cdm_galaxy_position_filtered = cdm_galaxy_position[cdm_galaxy_filtered_ind]
wdm_galaxy_position_filtered = wdm_galaxy_position[wdm_galaxy_filtered_ind]

### Find all the galaxies within 3 Mpc of each MW-sized galaxy

In [7]:
# create the trees
cdm_tree = spatial.cKDTree(cdm_galaxy_position_filtered)
wdm_tree = spatial.cKDTree(wdm_galaxy_position_filtered)

In [8]:
# create an empty array for the indices of the galaxies
cdm_3Mpc_sphere_galaxy = []

# iterate through each MW-sized galaxy
for i in trange(len(cdm_MW_galaxy_position)):
    
    # find the indices of all galaxies in a 3 Mpc radius around each MW-sized galaxy
    ind = cdm_tree.query_ball_point(cdm_MW_galaxy_position[i], 3)
    
    # record the indices for each MW-sized galaxy
    cdm_3Mpc_sphere_galaxy.append(ind)

100%|██████████████████████████████████████████████████████████████████████████████| 802/802 [00:00<00:00, 6637.05it/s]


In [9]:
# create an empty array for the indices of the galaxies
wdm_3Mpc_sphere_galaxy = []

# iterate through each MW-sized galaxy
for i in trange(len(wdm_MW_galaxy_position)):
    
    # find the indices of all galaxies in a 3 Mpc radius around each MW-sized galaxy
    ind = wdm_tree.query_ball_point(wdm_MW_galaxy_position[i], 3)
    
    # record the indices for each MW-sized galaxy
    wdm_3Mpc_sphere_galaxy.append(ind)

100%|██████████████████████████████████████████████████████████████████████████████| 703/703 [00:00<00:00, 7485.38it/s]


### Find all the galaxies within the Virial Radius of each MW-sized galaxy

In [ ]:
# create an empty array for the indices of the galaxies
cdm_VR_sphere_galaxy = []

# iterate through each MW-sized galaxy
for i in trange(len(cdm_MW_galaxy_position)):
    
    # find the indices of all galaxies in a 3 Mpc radius around each MW-sized galaxy
    ind = cdm_tree.query_ball_point(cdm_MW_galaxy_position[i], cdm_galaxy['R200'][:][cdm_MW_galaxy_ind[i]])
    
    # record the indices for each MW-sized galaxy
    cdm_VR_sphere_galaxy.append(ind)

In [ ]:
# create an empty array for the indices of the galaxies
wdm_VR_sphere_galaxy = []

# iterate through each MW-sized galaxy
for i in trange(len(wdm_MW_galaxy_position)):
    
    # find the indices of all galaxies in a 3 Mpc radius around each MW-sized galaxy
    ind = wdm_tree.query_ball_point(wdm_MW_galaxy_position[i], wdm_galaxy['R200'][:][wdm_MW_galaxy_ind[i]])
    
    # record the indices for each MW-sized galaxy
    wdm_VR_sphere_galaxy.append(ind)

### Find the indices of galaxies in between the virial radius of each halo and 3 Mpc

In [ ]:
# create an empty array for the indices of the galaxies that are not within the virial radius of each halo
cdm_shell_galaxy = []

# iterate through each MW-sized galaxy
for i in trange(len(cdm_3Mpc_sphere_galaxy)):
    
    # find the indices of all galaxies that are not within the virial radius of each halo
    ind = [j for j in cdm_3Mpc_sphere_galaxy[i] if j not in cdm_VR_sphere[i]]
    
    # record the indices for each MW-sized galaxy
    cdm_shell_galaxy.append(ind)

In [ ]:
# create an empty array for the indices of the galaxies that are not within the virial radius of each halo
wdm_shell_galaxy = []

# iterate through each MW-sized galaxy
for i in trange(len(wdm_3Mpc_sphere_galaxy)):
    
    # find the indices of all galaxies that are not within the virial radius of each halo
    ind = [j for j in wdm_3Mpc_sphere_galaxy[i] if j not in wdm_VR_sphere[i]]
    
    # record the indices for each MW-sized galaxy
    wdm_shell_galaxy.append(ind)

### Find their positions

# Find all halos that are as heavy as the Milky Way
### Access the masses and positions of CDM and WDM halos

In [10]:
# Access mass/position for wdm/cdm
print(list(cdm.keys()),list(wdm.keys()))

cdm_mass = cdm['SubhaloMass'][:]
cdm_position = cdm['SubhaloPos'][:]

wdm_mass = wdm['SubhaloMass'][:]
wdm_position = wdm['SubhaloPos'][:]

cdm_mass, cdm_position, wdm_mass, wdm_position

['IsCentral', 'R200', 'SubhaloMass', 'SubhaloPos'] ['IsCentral', 'R200', 'SubhaloMass', 'SubhaloPos']


(array([1.87617493e+14, 1.84468594e+14, 6.89978024e+13, ...,
        1.76011232e+08, 1.76011232e+08, 1.76011232e+08]),
 array([[10.93832207, 81.23078918, 54.77935028],
        [17.94562721, 79.86347961, 53.3843956 ],
        [15.62266159, 78.18474579, 52.83857346],
        ...,
        [ 1.47748184,  3.54455233, 99.53264618],
        [99.32190704,  2.56014371,  2.57368279],
        [99.0621109 ,  4.18381071,  2.89533401]]),
 array([1.89941540e+14, 1.86912732e+14, 6.83256610e+13, ...,
        1.76011232e+08, 1.76011232e+08, 1.76011232e+08]),
 array([[1.80150642e+01, 7.98655701e+01, 5.34328232e+01],
        [1.09048624e+01, 8.12610779e+01, 5.48179703e+01],
        [1.56323385e+01, 7.81665344e+01, 5.28157997e+01],
        ...,
        [6.02296066e+00, 5.05078554e+00, 9.79758301e+01],
        [5.75055540e-01, 3.26677370e+00, 3.52767438e-01],
        [2.07897183e-03, 3.23730040e+00, 9.97708917e-01]]))

### Find all central halos within the mass range $0.8-2*10^{12}$ M$_{\odot}$

In [11]:
# get the indices
cdm_MW_ind = np.where((cdm_mass>0.8*10**12) & (cdm_mass<2*10**12) & (cdm['IsCentral'][:]==1))[0]
wdm_MW_ind = np.where((wdm_mass>0.8*10**12) & (wdm_mass<2*10**12) & (wdm['IsCentral'][:]==1))[0]

len(cdm_MW_ind), len(wdm_MW_ind)

(1057, 1065)

In [12]:
# get the positions
cdm_MW_position = cdm_position[cdm_MW_ind]
wdm_MW_position = wdm_position[wdm_MW_ind]

### Find all the galaxies within 3 Mpc of each MW-sized halo

In [13]:
# create an empty array for the indices of the galaxies
cdm_3Mpc_sphere = []

# iterate through each MW-sized galaxy
for i in trange(len(cdm_MW_position)):
    
    # find the indices of all galaxies in a 3 Mpc radius around each MW-sized halo
    ind = cdm_tree.query_ball_point(cdm_MW_position[i], 3)
    
    # record the indices for each MW-sized halo
    cdm_3Mpc_sphere.append(ind)

100%|████████████████████████████████████████████████████████████████████████████| 1057/1057 [00:00<00:00, 6666.13it/s]


In [14]:
# create an empty array for the indices of the galaxies
wdm_3Mpc_sphere = []

# iterate through each MW-sized galaxy
for i in trange(len(wdm_MW_position)):
    
    # find the indices of all galaxies in a 3 Mpc radius around each MW-sized halo
    ind = wdm_tree.query_ball_point(wdm_MW_position[i], 3)
    
    # record the indices for each MW-sized halo
    wdm_3Mpc_sphere.append(ind)

100%|████████████████████████████████████████████████████████████████████████████| 1065/1065 [00:00<00:00, 6995.33it/s]


### Find all the galaxies within the Virial Radius of each MW-sized halo

In [16]:
# create an empty array for the indices of the galaxies
cdm_VR_sphere = []

# iterate through each MW-sized galaxy
for i in trange(len(cdm_MW_position)):
    
    # find the indices of all galaxies in a 3 Mpc radius around each MW-sized galaxy
    ind = cdm_tree.query_ball_point(cdm_MW_position[i], cdm['R200'][:][cdm_MW_ind[i]])
    
    # record the indices for each MW-sized halo
    cdm_VR_sphere.append(ind)

100%|██████████████████████████████████████████████████████████████████████████████| 1057/1057 [00:27<00:00, 38.06it/s]


In [17]:
# create an empty array for the indices of the galaxies
wdm_VR_sphere = []

# iterate through each MW-sized galaxy
for i in trange(len(wdm_MW_position)):
    
    # find the indices of all galaxies in a 3 Mpc radius around each MW-sized galaxy
    ind = wdm_tree.query_ball_point(wdm_MW_position[i], wdm['R200'][:][wdm_MW_ind[i]])
    
    # record the indices for each MW-sized halo
    wdm_VR_sphere.append(ind)

100%|██████████████████████████████████████████████████████████████████████████████| 1065/1065 [00:20<00:00, 52.51it/s]


### Find the indices of galaxies in between the virial radius of each halo and 3 Mpc

In [18]:
# create an empty array for the indices of the galaxies that are not within the virial radius of each halo
cdm_shell = []

# iterate through each MW-sized halo
for i in trange(len(cdm_3Mpc_sphere)):
    
    # find the indices of all galaxies that are not within the virial radius of each halo
    ind = [j for j in cdm_3Mpc_sphere[i] if j not in cdm_VR_sphere[i]]
    
    # record the indices for each MW-sized halo
    cdm_shell.append(ind)

100%|████████████████████████████████████████████████████████████████████████████| 1057/1057 [00:00<00:00, 5312.58it/s]


In [19]:
# create an empty array for the indices of the galaxies that are not within the virial radius of each halo
wdm_shell = []

# iterate through each MW-sized halo
for i in trange(len(wdm_3Mpc_sphere)):
    
    # find the indices of all galaxies that are not within the virial radius of each halo
    ind = [j for j in wdm_3Mpc_sphere[i] if j not in wdm_VR_sphere[i]]
    
    # record the indices for each MW-sized halo
    wdm_shell.append(ind)

100%|████████████████████████████████████████████████████████████████████████████| 1065/1065 [00:00<00:00, 7772.51it/s]


### Find their positions